In [27]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers , models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

from PIL import Image

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


In [28]:
train = pd.read_csv('./train_without_na.csv')
test = pd.read_csv('./test.csv')

In [29]:
train['Category'].unique()

array(['Men Tshirts', 'Sarees', 'Kurtis', 'Women Tshirts',
       'Women Tops & Tunics'], dtype=object)

In [30]:

category = ['Men Tshirts', 'Sarees', 'Kurtis', 'Women Tshirts',
       'Women Tops & Tunics']
target = [f'attr_{i}' for i in range(1 , 11)]

In [31]:
unique_class_in_each_target_label = []

In [32]:
for i in target:
    unique_class_in_each_target_label.append(train[i].nunique())

In [33]:
for i in unique_class_in_each_target_label:
    print(i)

18
14
10
19
14
8
12
11
14
9


In [34]:
# For Kaggle

train['image_path'] = train.apply(lambda x: './Data/train_images/' + x['image_path'].split('/')[-1], axis=1)
test['image_path'] = test.apply(lambda x: './Data/test_images/' + x['image_path'].split('/')[-1], axis=1)

In [35]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1),
])

In [36]:
def prepare_images(image_paths, image_size=(224, 224)):
    """
    Converts image paths into augmented tensors using predefined data augmentation layers.
    
    Args:
        image_paths (list of str): List of image paths.
        image_size (tuple): Target size of images (height, width).
    
    Returns:
        np.array: Array of preprocessed and augmented image tensors.
    """

    images = []
    
    for path in image_paths:
        try:
            img = Image.open(path).convert("RGB")
            img = img.resize(image_size)  
            img_array = np.array(img) / 255.0  
            
            # Add batch dimension and convert to tensor
            img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)
            img_tensor = tf.expand_dims(img_tensor, axis=0)  
            
            # Apply data augmentation
            augmented_img = data_augmentation(img_tensor)[0].numpy()  
            images.append(augmented_img)

        except Exception as e:
            print(f"Error loading image {path}: {e}")
    
    return np.array(images)


In [37]:
def predict_labels(unique_class_in_each_target_label, x_train, y_train, x_predict):
    """
    Train a ResNet50-based classifier and predict labels for multiple target labels.
    
    Args:
        unique_class_in_each_target_label (list): List of unique class counts for each target label.
        x_train (np.array): Preprocessed training images.
        y_train (list of np.array): List of labels for training images, one array per target.
        x_predict (np.array): Preprocessed images to predict.
    
    Returns:
        list of dict: List of dictionaries with structure {'id': <id>, 'label': <label>}.
    """
    
    # Convert each target label to categorical format
    y_train_categorical = [
        to_categorical(y_train[i], num_classes=unique_class_in_each_target_label[i])
        for i in range(len(unique_class_in_each_target_label))
    ]
    
    # Load the ResNet50 base model
    base_model = ResNet50(
        weights='./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False  
    
    # Define the model
    model = model(
        inputs=base_model.input,
        outputs=[
            Dense(num_classes, activation='softmax', name=f'target_{i+1}')(GlobalAveragePooling2D()(base_model.output))
            for i, num_classes in unique_class_in_each_target_label
        ]
    )

    # Compile the model
    model.compile(
        optimizer='adam', 
        loss=['categorical_crossentropy' for _ in unique_class_in_each_target_label], 
        metrics=['accuracy']
    )
    
    # Fit the model
    model.fit(
        x_train, 
        y_train_categorical, 
        epochs=4, 
        batch_size=32, 
        verbose=1
    )
    
    # Predict on the input
    predictions = model.predict(x_predict)
    y_predict = [np.argmax(pred, axis=1) for pred in predictions]
    
    return y_predict



In [ ]:
predict_df = test[['id' , 'image_path']]
train_df = train[['id' , 'image_path' , 'attr_1','attr_2','attr_3','attr_4','attr_5','attr_6','attr_7','attr_8','attr_9','attr_10']]

le = LabelEncoder()

for i in target:
    train_df[i] = le.fit_transform(train_df[i])

class_to_label = dict(zip(le.classes_, range(len(le.classes_))))
label_to_class = {v:k  for k, v in class_to_label.items()}

y_train = train_df[target]

x_train = prepare_images(train_df['image_path'])
x_predict = prepare_images(predict_df['image_path'])

y_predict = predict_labels(
    unique_class_in_each_target_label,
    x_train,
    y_train,
    x_predict
)
        
# Step 3: Prepare final result
predict_df['label'] = [label_to_class[label] for label in y_predict]
result = predict_df[['id', 'label']]

result.to_csv(f'submission.csv', index=False)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22884\3963779241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[i] = le.fit_transform(train_df[i])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22884\3963779241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[i] = le.fit_transform(train_df[i])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22884\3963779241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

Error loading image ./Data/train_images/040389.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image ./Data/train_images/040394.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image ./Data/train_images/040395.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image ./Data/train_images/040396.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image ./Data/train_images/040397.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image ./Data/train_images/040398.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image ./Data/train_images/040399.jpg: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image 